# Breast Cancer Detection With Mammograms

This is a demonstration on how to build a model that gives prediction on breast cancer given a set of mammograms.

The dataset used is the [Augmented INBreast Dataset](https://www.kaggle.com/datasets/eoussama/breast-cancer-mammograms).

A mirror repository for this notebook can be found on [github](https://github.com/gomu-gomu/ma-dl-projet-1).


## Dependencies

In [ ]:
!pip install tensorflow==2.12.0

In [ ]:
import os
import shutil
import pickle
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import keras
from keras.callbacks import CSVLogger
from keras.models import Sequential, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

## Dataset INBreast Dataset
We have to prepare our dataset and split it in a convenient way. First we start by setting up separate folders fore each sub-category.

In [ ]:
output_path = "/kaggle/working"
input_path = "/kaggle/input/breast-cancer-mammograms/"
data_path = f"{output_path}/split_data"

for split in ['train', 'val', 'test']:
    for cls in ['benign', 'malignant']:
        os.makedirs(os.path.join(data_path, split, cls), exist_ok=True)

def get_files(cls):
    return [f for f in os.listdir(os.path.join(input_path, cls)) 
           if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

benign_files = get_files('benign')
malignant_files = get_files('malignant')

def split_data(files):
    train, temp = train_test_split(files, test_size=0.2, random_state=42)
    val, test = train_test_split(temp, test_size=0.5, random_state=42)
    return train, val, test

benign_train, benign_val, benign_test = split_data(benign_files)
malignant_train, malignant_val, malignant_test = split_data(malignant_files)

def copy_files(files, cls, split):
    for f in files:
        src = os.path.join(input_path, cls, f)
        dst = os.path.join(data_path, split, cls, f)
        shutil.copy(src, dst)

copy_files(benign_train, 'benign', 'train')
copy_files(benign_val, 'benign', 'val')
copy_files(benign_test, 'benign', 'test')

copy_files(malignant_train, 'malignant', 'train')
copy_files(malignant_val, 'malignant', 'val')
copy_files(malignant_test, 'malignant', 'test')

print("Dataset split completed!")

### Loading the data
Then we create an image data generator that will scale each pixel's value by 1/255, essentially normalizing them from the standard 0–255 range to a 0–1 range. Then we load the three data collections separately.

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)

train = datagen.flow_from_directory(
    f'{data_path}/train',
    target_size=(224, 224),
    class_mode='binary',
    batch_size=64
)

val = datagen.flow_from_directory(
    f'{data_path}/val',
    target_size=(224, 224),
    class_mode='binary',
    batch_size=64
)

test = datagen.flow_from_directory(
    f'{data_path}/test',
    target_size=(224, 224),
    class_mode='binary',
    batch_size=64
)

### Data inspection
Now we fetch the first batch of images and labels out of `train`, and confirm their shape.

In [ ]:
imgs, labels = next(train)
imgs.shape

* **64**: the batch size (number of images).
* **224 × 224**: the height and width of each image, as defined in `target_size=(224, 224)`.
* **3**: the number of color channels (RGB).

We confirm that we only have 2 classes. **Benign** and **Malgnant**.

In [ ]:
train.class_indices

Displaying the first image in the training dataset should show us a grayscale scan of a breast, accompanied with the correct class decided by its diagnosis.

In [ ]:
plt.imshow(imgs[0])
print(f'Class: {labels[0]}')

The class is `1.0`, which matches **Malignant** on the class indices of the training dataset.

## Model

We're gonna build a simple Convolutional Neural Network (CNN) for binary classification. First, we add three convolution-and-pooling blocks to extract spatial features from the images. Then, we flatten the feature maps and pass them through a couple of dense (fully connected) layers, including a dropout for regularization. Finally, we use a single output neuron with a sigmoid activation for predicting whether an image is benign or malignant.

* Step 1:

    - We start by initializing a new `Sequential` model, which is a linear stack of layers in Keras.
* Step 2:

    - `Conv2D(32, (3, 3))`: A convolutional layer with 32 filters/kernels, each 3×3 in size.
    - `input_shape=(224,224,3)`: This is the expected shape of the input images. 224×224 resolution, 3 color channels (RGB).
    - `activation='relu'`: Uses the ReLU activation function.
    - `MaxPooling2D(pool_size=(2, 2))`: Reduces the spatial dimensions by taking the maximum value in each 2×2 pool region, effectively halving the height and width.
* Step 3:

    - Similar to the first block, but it doesn't need an input_shape since Keras automatically infers the shape from the previous layer's output.
    - Another 32 filters, each 3×3, followed by a 2×2 max pool to further downsample.
* Step 4:

    - Now we increase the number of filters to 64. A larger number of filters can learn more complex features.
    - Followed by another 2×2 max pool for further spatial reduction.
* Step 5:

    - `Flatten()`: Converts the 3D feature maps (height × width × channels) into a 1D vector, so it can be fed into a dense (fully connected) layer.
    - `Dense(64, activation='relu')`: A fully connected layer with 64 neurons, learning high-level combinations of features.
    - `Dropout(0.5)`: Randomly sets 50% of the neurons to zero during training to reduce overfitting.
    - `Dense(1, activation='sigmoid')`: The final output layer. Since this is a binary classification (benign vs. malignant), we use a single neuron with a sigmoid activation, which outputs a probability between 0 and 1.
* Step 6:

    - `loss='binary_crossentropy'`: The appropriate loss function for binary classification.
    - `optimizer='rmsprop'`: A gradient-based optimization algorithm (alternative to Adam, SGD, etc.).
    - `metrics=['accuracy']`: We track accuracy during training/evaluation.

In [ ]:
from tensorflow.keras.metrics import AUC, Precision, Recall


# Step 1 - Initializing model
model = Sequential()

# Step 2 - First Convolution + MaxPooling
model.add(Conv2D(32, (3, 3), input_shape=(224,224,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Step 3 - Second Convolution + MaxPooling
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Step 4 - Third Convolution + MaxPooling
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Step 5 - Flatten + Dense + Dropout + Output
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Step 6 - Compile
# Step 6 - Compile


# additional metrics
model.compile(loss='binary_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy', AUC(name='auc'), Precision(name='precision'), Recall(name='recall')])
model.summary()

- **Output Shape**: The size of the activation maps after each layer.

- **Param #**: The number of trainable parameters (weights and biases).

> Notice that most of the parameters (over 2.7 million) are in the fully connected layer (dense_6), which converts the flattened feature maps to 64 neurons.

- **Total params**: The sum of all parameters in the network.

## Training

In this step we will train the CNN for 100 epochs while logging metrics to a CSV file (training.log) for analysis, and then saves the trained model for future use.

In [ ]:


# Calculating how many batches (steps) makeup one full pass (epoch) through our training and validation datasets.
STEP_SIZE_TRAIN = train.n // train.batch_size
STEP_SIZE_VAL = val.n // val.batch_size

os.makedirs(os.path.join(output_path, 'logs'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'out'), exist_ok=True)

# This callback records loss, accuracy, and other metrics for each epoch into a CSV file named training.log.
# It's helpful for tracking and comparing training progress over time.
csv_logger = CSVLogger(f'{output_path}/logs/training.log', separator=',', append=False)

history = model.fit_generator(generator = train, 
                    steps_per_epoch=STEP_SIZE_TRAIN, 
                    validation_data=test,
                    validation_steps=STEP_SIZE_VAL,
                    epochs = 50,
                    # epochs=100,
                    callbacks=[csv_logger])










model.save(f'{output_path}/out/model.h5')



To determine the step sizes, take note of the following:






- `train.n` and `val.n`: The total number of samples in each dataset.
- `train.batch_size` and `val.batch_size`: The batch size.
- Using integer division `//` gives you the number of batches needed.

As for fitting the model:
- `model.fit_generator(...)`: Trains the model using the data generated from train (the training image generator) in batches.
- `steps_per_epoch=STEP_SIZE_TRAIN`: How many steps (batches) to run per epoch during training.
- `validation_data=test` and `validation_steps=STEP_SIZE_VAL`: Here, you're using the test generator for validation, with STEP_SIZE_VAL batches per epoch.
- `epochs=100`: The number of times the model will see the entire dataset.
- `callbacks=[csv_logger]`: Logs each epoch's metrics to the CSV file.

We can visualize how our model's accuracy changes over the course of training. By plotting both the training and validation accuracy, we'll get a clear picture of how well the model is fitting to the training data and how effectively it generalizes to unseen data.

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})

plt.plot(np.arange(1, len(history.history['accuracy'])+1,1), history.history['accuracy'], color='navy', label = 'Accuracy')
plt.plot(np.arange(1, len(history.history['accuracy'])+1,1), history.history['val_accuracy'], color='red', label='Validation Accuracy')
plt.legend()

The plot shows two lines: one for the training **accuracy** (in navy) and another for the validation accuracy (in red). As the number of epochs increases, we can observe whether the model converges, if it overfits (training accuracy outpacing validation accuracy), or if both accuracies improve steadily over time. This helps us assess the model's performance and decide on further tuning.

From the plot, we can see that the training accuracy (blue line) reaches near-perfect levels, while the validation accuracy (red line) plateaus around the high 80s to low 90s. This suggests the model is learning effectively but also overfitting somewhat—its performance on unseen data (validation accuracy) isn't as high as on the training set. Even so, the validation accuracy still remains robust, indicating that the model generally performs well at classifying new examples despite not matching the near-perfect training performance.

We can do the same with the saved model. We can read back the CSV file (training.log) that was generated by the `CSVLogger` during training. Each row in this file corresponds to an epoch, and the columns contain metrics like loss, accuracy, validation loss, and validation accuracy.

In [ ]:
log_data = pd.read_csv(f'{output_path}/logs/training.log', sep=',', engine='python')
log_data

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})

plt.plot(np.arange(1, len(log_data['accuracy'])+1,1), log_data['accuracy'], color='navy', label = 'Accuracy')
plt.plot(np.arange(1, len(log_data['accuracy'])+1,1), log_data['val_accuracy'], color='red', label='Validation Accuracy')
plt.legend()

Again, visualizing how our model's **training loss** (in navy) compares to the **validation loss** (in red) over each epoch of training. By charting both curves, we can spot whether the model is successfully generalizing (both losses decreasing together) or if it begins to overfit (training loss keeps going down while validation loss rises). This step is essential to ensure our model is learning appropriately and to guide further tuning.

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})

plt.plot(np.arange(1, len(history.history['loss'])+1,1), history.history['loss'], color='navy', label = 'Loss')
plt.plot(np.arange(1, len(history.history['loss'])+1,1), history.history['val_loss'], color='red', label='Validation Loss')
plt.legend()

From this plot, it's clear that the **training loss** (blue) steadily goes down to very low values, indicating the model is fitting the training data extremely well. However, the **validation loss** (red) varies up and down rather than following the same steady downward trend, suggesting the model is **overfitting** and not generalizing as consistently to unseen data. The spikes in the validation loss can occur for various reasons (e.g., data variability, small validation sets, or model instability), but the key takeaway is that while the model memorizes the training set effectively, its performance on new data is less stable.

Doing the same again with the logged data.

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})

plt.plot(np.arange(1, len(log_data['loss'])+1,1), log_data['loss'], color='navy', label = 'Loss')
plt.plot(np.arange(1, len(log_data['loss'])+1,1), log_data['val_loss'], color='red', label='Validation Loss')
plt.legend()

In [ ]:
# print classification report before hibernation 
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Step 1: Load the trained model
from tensorflow.keras.models import load_model
model = load_model(f'{output_path}/out/model.h5')

# Step 2: Prepare the test data generator
test_generator = test  # Your existing test data generator

# Step 3: Predict the probabilities on the test set
# Note: Use predict() with batch size as needed
pred_probs = model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size + 1)
# For binary classification, convert probabilities to class labels
pred_labels = (pred_probs > 0.5).astype(int).flatten()

# Step 4: Get true labels
true_labels = test_generator.classes

# Step 5: Generate classification report
report = classification_report(true_labels, pred_labels, target_names=['benign', 'malignant'])
print("Classification Report:\n", report)

# Optional: Confusion matrix
cm = confusion_matrix(true_labels, pred_labels)
print("Confusion Matrix:\n", cm)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plotting the confusion matrix
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['benign', 'malignant'], yticklabels=['benign', 'malignant'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Normalize the confusion matrix to percentages
cm_percentage = (cm.astype('float') / cm.sum()) * 100

# Plotting the confusion matrix as percentages
plt.figure(figsize=(8,6))
sns.heatmap(cm_percentage, annot=True, fmt='.2f', cmap='Blues', xticklabels=['benign', 'malignant'], yticklabels=['benign', 'malignant'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix in Percentages')
plt.show()

In [ ]:
fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"AUC = {auc_score:.2f}")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Plot AUC
plt.figure(figsize=(10, 6))
plt.plot(history.history['auc'], label='Train AUC')
plt.plot(history.history['val_auc'], label='Val AUC')
plt.title('AUC over Epochs')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.legend()
plt.grid(True)
plt.savefig("AUC_Epochs.png", dpi=300)
plt.show()

# Plot Precision
plt.figure(figsize=(10, 6))
plt.plot(history.history['precision'], label='Train Precision')
plt.plot(history.history['val_precision'], label='Val Precision')
plt.title('Precision over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend()
plt.grid(True)
plt.savefig("Precision_Epochs.png", dpi=300)
plt.show()

# Plot Recall
plt.figure(figsize=(10, 6))
plt.plot(history.history['recall'], label='Train Recall')
plt.plot(history.history['val_recall'], label='Val Recall')
plt.title('Recall over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.legend()
plt.grid(True)
plt.savefig("Recall_Epochs.png", dpi=300)
plt.show()

# Plot Accuracy
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.savefig("Accuracy_Epochs.png", dpi=300)
plt.show()


In [ ]:
from sklearn.metrics import (
    precision_score, recall_score, f1_score, roc_auc_score, roc_curve
)
import numpy as np
import matplotlib.pyplot as plt

# Predict on all validation data
y_pred_prob = model.predict(test, verbose=1)

# Class prediction (binary threshold)
y_pred_class = (y_pred_prob > 0.5).astype(int).flatten()

# Ground truth labels
y_true = test.classes  # length = test.n

# Ensure alignment (optional but safe)
assert len(y_true) == len(y_pred_class)

# Metrics
precision = precision_score(y_true, y_pred_class)
recall = recall_score(y_true, y_pred_class)
f1 = f1_score(y_true, y_pred_class)
auc_score = roc_auc_score(y_true, y_pred_prob)

print(f"Test Precision: {precision:.4f}")
print(f"Test Recall:    {recall:.4f}")
print(f"Test F1 Score:  {f1:.4f}")
print(f"Test AUC:       {auc_score:.4f}")


In [ ]:
fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"AUC = {auc_score:.2f}")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()


## Hibernation

We can load our previously saved model from disk, then extract all the images and labels from our test generator into NumPy arrays. This prepares our entire test set in a convenient format for further evaluation or predictions.

In [ ]:
# Loading the previously saved model from the specified path. This includes the architecture, weights, and training configuration (if any).
restored_model = load_model(f'{output_path}/out/model.h5')

# Calculating how many batches you need to go through in order to cover our entire test set
steps = test.n//test.batch_size

# Reseting the test generator to start yielding batches from the first image again. This is useful if the generator's internal index was advanced by previous calls (e.g., during validation).
test.reset()

# Collecting the test data
X_test, y_test = [], []
for i in range(steps):
    a , b = test.next()
    X_test.extend(a) 
    y_test.extend(b)

# Converting lists to NumPy arrays
X_test, y_test = np.array(X_test), np.array(y_test)

We can save our test data arrays (X_test and y_test) to disk using `pickle.dump` and then reload them with `pickle.load`.

In [ ]:
# Pickling the model
pickle.dump(X_test, open(f'{output_path}/out/X_test.pkl', 'wb'))
pickle.dump(y_test, open(f'{output_path}/out/y_test.pkl', 'wb'))

# Reloading the model to ensure not thing is corrupt
X_test = pickle.load(open(f'{output_path}/out/X_test.pkl', 'rb'))
y_test = pickle.load(open(f'{output_path}/out/y_test.pkl', 'rb'))

print(X_test.shape, y_test.shape)

- `pickle.dump(...)`: Serializes (saves) Python objects to a file in a binary format.
- `'wb'`: Means write binary.
- `pickle.load(...)`: Deserializes (loads) the data back from the pickle files into memory.
- `'rb'`: Means read binary.

Result:

- **(704, 224, 224, 3)**: There are 704 images, each 224×224 pixels in size with 3 color channels (RGB).
- **(704,)**: The corresponding labels array for these images has 704 labels.

## Evaluation

We can evaluate the trained model on the full test dataset (`X_test`, `y_test`) and print its performance metrics. Specifically, we `model.evaluate(...)` to obtain the final **loss** and **accuracy** on unseen data, giving us a measure of how well our model generalizes beyond the training set.

### Evaluation the model

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

The above function runs the model on the test data (`X_test` and corresponding labels `y_test`) and returns a list (or tuple) of metrics. In this case, because our model was compiled with `loss='binary_crossentropy'` and `metrics=['accuracy']`, `score[0]` will be the test loss and `score[1]` will be the test accuracy. `verbose=0` means it will not print any progress bar or additional information during evaluation.

Test loss is about **0.48672**, which is a moderate number (lower is typically better). Test accuracy is about **0.97727**, suggesting our model correctly classifies around **97%** of the test samples. This indicates a solid performance on this binary classification task.

### Evaluating the saved model

In [ ]:
score = restored_model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Evidently, the saved model would yeild the same results.

## Prediction

We can use our trained model to generate predicted probabilities for each image in the test set. Since this is a binary classification problem, each number represents the model's estimated likelihood that the image belongs to class **1** (the **Malignant** class).

### Prediction with the model

In [ ]:
y_pred_prob = model.predict(X_test)
print(y_pred_prob)

### Prediction with the saved model

In [ ]:
y_pred_prob = restored_model.predict(X_test)
print(y_pred_prob)

Let's plot two sets of points for each test sample (indexed on the x-axis):

- `y_pred_prob` (red dots): These are the predicted probabilities for each sample. A value near 1 indicates the model believes the image is likely in class **1** (**Malignant**), and a value near 0 indicates the model thinks it's class **0** (**Bengin**).
- `y_test` (blue dots): These are the actual labels (ground truth), which are 0 or 1 in a binary classification. Here, each blue dot at the top represents an actual label of "1" while each blue dot along the bottom represents "0"

In [ ]:
plt.plot(y_pred_prob,'.', color='red', label='Predicted Probabilty')
plt.plot(y_test, '.', color='navy', label='Actual Labels')
plt.xlabel('Instance Number')
plt.ylabel('Probability')
plt.legend()

## Thresholding

Inspecting the Ground Truth Labels:

In [ ]:
y_test

Let's apply a threshold of 0.5 on the model's predicted probabilities to get final class predictions `y_pred`.

In [ ]:
threshold = 0.5

y_pred = np.where(y_pred_prob > threshold, 1 ,0)
y_pred.squeeze

y_pred

- `threshold = 0.5`: We choose 0.5 as the cutoff. If the predicted probability `y_pred_prob` is above 0.5, we predict class 1, otherwise class 0. This is the standard threshold for binary classification when using a sigmoid output layer.
- `np.where(y_pred_prob > threshold, 1, 0)`: Converts the continuous probabilities into discrete class predictions (0 or 1).
- `y_pred.squeeze()`: Removes extra dimensions if present (e.g., if `y_pred` is of shape (N, 1) instead of (N,)).
- `y_pred`: Now contains integer class predictions for each sample—either 0 or 1.

## Analysis

We can visualize the Confusion Matrix as a Heatmap to help us quickly assess the performance of your model, whether it's correctly predicting malignant and benign cases, and how often it makes misclassifications in each direction.

In [ ]:
sns.set(rc={'figure.figsize':(7.7,6.27)})
sns.heatmap(confusion_matrix(y_test, y_pred), cmap=plt.cm.Blues, annot=True, annot_kws={"size": 32}, fmt='g')

plt.xticks([0.50,1.50], ['Malignant','Benign'], fontsize=20)
plt.yticks([0.50,1.50],['Malignant','Benign'], fontsize=20)

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title('Confusion Matrix for Breast Cancer')

In [ ]:
# confusion matrix in percentage 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Compute the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Normalize the confusion matrix to get percentages
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Plot the heatmap with normalized percentages
sns.set(rc={'figure.figsize':(7.7,6.27)})
sns.heatmap(cm_normalized, cmap=plt.cm.Blues, annot=True, fmt='.2%', annot_kws={"size": 32})

# Set tick labels with font size
plt.xticks([0.5, 1.5], ['Malignant', 'Benign'], fontsize=20)
plt.yticks([0.5, 1.5], ['Malignant', 'Benign'], fontsize=20)

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title('Confusion Matrix for Breast Cancer in Percentage')
plt.show()

In [ ]:
from sklearn.metrics import classification_report
import pandas as pd

# Generate the classification report as a dictionary
report_dict = classification_report(y_test, y_pred, target_names=['Benign', 'Malignant'], output_dict=True)

# Convert the report into a pandas DataFrame
report_df = pd.DataFrame(report_dict).transpose()

# Select metrics for classes
metrics = report_df.loc[['Benign', 'Malignant'], ['precision', 'recall', 'f1-score', 'support']]

# Extract macro and weighted averages
macro_avg = report_df.loc['macro avg', ['precision', 'recall', 'f1-score', 'support']]
weighted_avg = report_df.loc['weighted avg', ['precision', 'recall', 'f1-score', 'support']]

# Append macro and weighted averages to the metrics DataFrame
metrics.loc['Macro Avg'] = macro_avg
metrics.loc['Weighted Avg'] = weighted_avg

# Print the table
print(metrics)



print("retry for 100 epochs")

From the above, we see that out of **237 malignant** samples (top row), the model correctly identifies **229** as malignant but misclassifies **8** as benign. For the **467** benign samples (bottom row), **459** are correctly predicted as benign, with **8** mistakenly labeled as malignant. Overall, the model does a good job classifying benign cases (low false positives) but still misses some malignant cases (8 false negatives). In a medical context, those false negatives are critical because they represent malignant tumors misdiagnosed as benign. Nonetheless, the overall accuracy is high, indicating strong performance on the dataset.

In [ ]:
print(classification_report(y_test, y_pred, target_names = ['Benign (Class 0)','Malignant (Class 1)']))

- **Precision**: Measures how many of the samples predicted as a certain class (e.g., **Malignant**) actually belong to that class.
- **Recall**: (Also known as sensitivity) measures how many of the samples belonging to a certain class (e.g., Malignant) are correctly identified.
- **F1 score**: The harmonic mean of precision and recall. It balances both metrics into one number.
- **Support**: The number of samples in the dataset belonging to each class.
- **Accuracy**: The proportion of all samples (both positive and negative) that were correctly classified.
- **Macro avg**: The metric (e.g., precision, recall, F1) independently for each class, and then takes the average. This treats all classes equally.
- **Weighted avg**: The metric (e.g., precision, recall, F1) for each class and weights them by the number of samples from that class.​

The model does particularly well identifying malignant tumors (high recall of 0.98). This is typically desirable in a medical setting, because missing malignant cases (false negatives) can be more critical. However, the recall for benign cases is lower (0.97), meaning some benign samples are misclassified as malignant. Overall, the model is still quite strong in distinguishing the two classes, as shown by an F1-score above 0.98 for malignant and 0.97 for benign.

If we generate an ROC curve by comparing the model's predicted probabilities `y_pred_prob` against the true labels `y_test` at various decision thresholds we can calculate the false positive rate (FPR) and true positive rate (TPR) for each threshold and then plot them.

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
area_under_curve = auc(fpr, tpr)

plt.plot([0, 1], [0, 1], 'r--')
plt.plot(fpr, tpr, label='AUC = {:.3f}'.format(area_under_curve))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')




plt.savefig("ROC_Curve.png")


plt.show()




In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Predict probabilities on train and validation sets
y_train_prob = model.predict(train, verbose=1)
y_val_prob = model.predict(test, verbose=1)

# Get true binary labels
y_train_true = train.classes
y_val_true = test.classes

# Compute ROC curve and AUC for training
fpr_train, tpr_train, _ = roc_curve(y_train_true, y_train_prob)
auc_train = auc(fpr_train, tpr_train)

# Compute ROC curve and AUC for validation
fpr_val, tpr_val, _ = roc_curve(y_val_true, y_val_prob)
auc_val = auc(fpr_val, tpr_val)

# Plot
plt.figure(figsize=(10, 7))
plt.plot(fpr_train, tpr_train, label=f"Train AUC = {auc_train:.4f}", color='blue')
plt.plot(fpr_val, tpr_val, label=f"Validation AUC = {auc_val:.4f}", color='green')

plt.plot([0, 1], [0, 1], 'k--', label="Random Classifier")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate", fontsize=14)
plt.ylabel("True Positive Rate", fontsize=14)
plt.title("ROC Curve for Train vs Validation", fontsize=16)
plt.legend(loc="lower right", fontsize=12)
plt.grid(True)
plt.show()


The AUC (Area Under the Curve) of 0.947 shows that our model has strong discriminatory power—it can distinguish malignant from benign cases accurately across a broad range of thresholds. The closer the AUC is to 1.0, the better the model's overall performance.

Now, let's sample 25 random test images from the dataset `X_test`. For each image we'll retrieve:

1. The predicted class (either 0 or 1) and mapped it to "Benign" or "Malignant".
2. The model's probability for that predicted class.

In [ ]:
# mapping numeric predictions (0 or 1) to the corresponding string labels.
cancer_labels = ['Benign', 'Malignant']

# Randomly picking 25 indices from the test set, then retrieving those images.
img_indices = np.random.randint(0, len(X_test), size=25)
sample_test_images = X_test[img_indices]

# Creatinng a list of predicted labels (e.g., "Benign" or "Malignant") corresponding to each of the chosen images.
actual_test_labels = [cancer_labels[int(label)] for label in y_test[img_indices]]

max_prediction = np.max(y_pred_prob, axis=1)
prediction_probs = np.max(y_pred_prob, axis=1)

- `max_prediction`: For each test sample, takes the index of the highest probability. (In a binary classification with one output neuron, this step usually isn't necessary—if you had two output neurons, it picks whichever neuron has the higher probability.)
- `prediction_probs`: Grabs the actual probability value of that highest-probability class.

We then display each image in a subplot, labeling it with the predicted class and its predicted probability on the x-axis, and (in this snippet) also showing the predicted label again on the y-axis.

In [ ]:
plt.figure(figsize=(15,15))

for i, (img, pred_idx, prob, true_label) in enumerate(
    zip(sample_test_images, max_prediction[img_indices], 
        prediction_probs[img_indices], actual_test_labels)
):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)

    plt.imshow(img)
    plt.xlabel(f"{cancer_labels[int(round(pred_idx))]} ({prob:.3f})")
    plt.ylabel(true_label)

plt.tight_layout()
plt.show()

By visually inspecting these images, you can see how confident the model is for each prediction. If "Benign" images mostly have predicted probabilities close to 1.0 for "Benign", and "Malignant" images similarly have high probabilities for "Malignant" the model is performing well. In the provided screenshot, most if not all predicted labels and confidence levels appear correct or strongly confident. However, any discrepancies (e.g., a visually "Malignant" image labeled as "Benign" with a high probability) would signal a misclassification worth investigating.